# Classify structured data using Keras preprocessing layers

https://www.tensorflow.org/tutorials/structured_data/preprocessing_layers

## Setup

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [2]:
# Setup memory to fix critical issue
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Use Pandas to create a dataframe

In [4]:
import pathlib

dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'
csv_file = 'datasets/petfinder-mini/petfinder-mini.csv'

tf.keras.utils.get_file('petfinder_mini.zip', dataset_url,
                        extract=True, cache_dir='.')
dataframe = pd.read_csv(csv_file)

In [5]:
dataframe.columns

Index(['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize',
       'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'Description',
       'PhotoAmt', 'AdoptionSpeed'],
      dtype='object')

In [6]:
dataframe.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,2


In [7]:
dataframe.dtypes

Type             object
Age               int64
Breed1           object
Gender           object
Color1           object
Color2           object
MaturitySize     object
FurLength        object
Vaccinated       object
Sterilized       object
Health           object
Fee               int64
Description      object
PhotoAmt          int64
AdoptionSpeed     int64
dtype: object

In [8]:
dataframe['Age']=dataframe['Age'].astype(np.int32)
dataframe['Fee']=dataframe['Fee'].astype(np.int32)
dataframe['PhotoAmt']=dataframe['PhotoAmt'].astype(np.int32)
dataframe['AdoptionSpeed']=dataframe['AdoptionSpeed'].astype(np.int32)

In [9]:
dataframe.dtypes

Type             object
Age               int32
Breed1           object
Gender           object
Color1           object
Color2           object
MaturitySize     object
FurLength        object
Vaccinated       object
Sterilized       object
Health           object
Fee               int32
Description      object
PhotoAmt          int32
AdoptionSpeed     int32
dtype: object

In [10]:
my_array=np.array([[1,2,3],[1,5,6],[1,5,7]])

df = pd.DataFrame(my_array, columns = ['Column_A','Column_B','Column_C'])

In [11]:
np.where(df['Column_A']==1, 0, 1)

array([0, 0, 0])

### Create target variable

In [12]:
# In the original dataset "4" indicates the pet was not adopted.
dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)

# Drop un-used columns.
dataframe = dataframe.drop(columns=['AdoptionSpeed', 'Description'])

### Split the dataframe into train, validation, and test

In [13]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

7383 train examples
1846 validation examples
2308 test examples


### Input pipeline df_to_dataset

If you were working with a very large CSV file (so large that it does not fit into memory), you would use `tf.data` to read it from disk directly. That is not covered in this tutorial.

In [14]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size = len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [34]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)

In [39]:
train_ds

<BatchDataset shapes: ({Type: (None,), Age: (None,), Breed1: (None,), Gender: (None,), Color1: (None,), Color2: (None,), MaturitySize: (None,), FurLength: (None,), Vaccinated: (None,), Sterilized: (None,), Health: (None,), Fee: (None,), PhotoAmt: (None,)}, (None,)), types: ({Type: tf.string, Age: tf.int32, Breed1: tf.string, Gender: tf.string, Color1: tf.string, Color2: tf.string, MaturitySize: tf.string, FurLength: tf.string, Vaccinated: tf.string, Sterilized: tf.string, Health: tf.string, Fee: tf.int32, PhotoAmt: tf.int32}, tf.int32)>

In [35]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of ages:', train_features['Age'])
print('A batch of targets:', label_batch)

Every feature: ['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt']
A batch of ages: tf.Tensor([4 1 8 2 3], shape=(5,), dtype=int32)
A batch of targets: tf.Tensor([1 1 1 1 0], shape=(5,), dtype=int32)


In [42]:
train_features

{'Type': <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'Dog', b'Dog', b'Dog', b'Cat', b'Cat'], dtype=object)>,
 'Age': <tf.Tensor: shape=(5,), dtype=int32, numpy=array([4, 1, 8, 2, 3])>,
 'Breed1': <tf.Tensor: shape=(5,), dtype=string, numpy=
 array([b'Mixed Breed', b'Mixed Breed', b'German Shepherd Dog',
        b'Russian Blue', b'Chausie'], dtype=object)>,
 'Gender': <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'Male', b'Male', b'Female', b'Female', b'Male'], dtype=object)>,
 'Color1': <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'Black', b'Brown', b'Black', b'Gray', b'Brown'], dtype=object)>,
 'Color2': <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'Brown', b'No Color', b'Brown', b'White', b'Golden'], dtype=object)>,
 'MaturitySize': <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'Medium', b'Medium', b'Medium', b'Medium', b'Small'], dtype=object)>,
 'FurLength': <tf.Tensor: shape=(5,), dtype=string, numpy=array([b'Short', b'Short', b'Medium', b'Short',

### Understand the input pipeline

In [ ]:
for feature_batch, label_batch in train_ds.take(1):
    print('Every feature:', list(feature_batch.keys()))
    print('A batch of ages:', feature_batch['Age'])
    print('A batch of targets:', label_batch)

### Demonstrate several types of feature columns

In [ ]:
example_batch = next(iter(train_ds))[0]

In [ ]:
next(iter(train_ds))[0]

In [ ]:
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

### Numeric columns

In [ ]:
photo_count = feature_column.numeric_column('PhotoAmt')

### Bucketized columns

In [ ]:
age = feature_column.numeric_column('Age')
print(age)
age_buckets = feature_column.bucketized_column(age, boundaries=[1, 3, 5])
print(demo(age_buckets))

### Categorical columns

In [ ]:
animal_type = feature_column.categorical_column_with_vocabulary_list('Type', ['Cat', 'Dog'])
animal_type_one_hot = feature_column.indicator_column(animal_type)
demo(animal_type_one_hot)

In [ ]:
animal_type.dtype

### Embedding columns

ATTENTION: it requires conda env in windows to avoid crash

In [ ]:
breed1 = feature_column.categorical_column_with_vocabulary_list('Breed1', dataframe.Breed1.unique())
breed1_embedding = feature_column.embedding_column(breed1, dimension=8)
demo(breed1_embedding)

### Hashed feature columns

In [ ]:
breed1_hashed = feature_column.categorical_column_with_hash_bucket('Breed1', hash_bucket_size = 10)
demo(feature_column.indicator_column(breed1_hashed))

### Crossed feature columns

In [ ]:
# OverflowError: Python int too large to convert to C long
crossed_feature = feature_column.crossed_column([age_buckets, animal_type], hash_bucket_size=1)
layers.DenseFeatures(feature_column.indicator_column(crossed_feature))

### Choose which columns to use

In [ ]:
feature_columns = []

for header in ['PhotoAmt', 'Fee', 'Age']:
    feature_columns.append(feature_column.numeric_column(header))

In [ ]:
# bucketized cols
age = feature_column.numeric_column('Age')
age_buckets = feature_column.bucketized_column(age, boundaries=[1,2,3,4,5])
feature_columns.append(age_buckets)

In [ ]:
# indicator_columns
indicator_column_names = ['Type', 'Color1', 'Color2', 'Gender', 'MaturitySize',
                          'FurLength', 'Vaccinated', 'Sterilized', 'Health']
for col_name in indicator_column_names:
    categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, dataframe[col_name].unique())
    indicator_column = feature_column.indicator_column(categorical_column)
    feature_columns.append(indicator_column)

In [ ]:
# embedding columns
breed1 = feature_column.categorical_column_with_vocabulary_list(
      'Breed1', dataframe.Breed1.unique())
breed1_embedding = feature_column.embedding_column(breed1, dimension=8)
feature_columns.append(breed1_embedding)

In [ ]:
# crossed columns
age_type_feature = feature_column.crossed_column([age_buckets, animal_type], hash_bucket_size=100)
feature_columns.append(feature_column.indicator_column(age_type_feature))

### Create a feature layer

In [ ]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [ ]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

## Create and train model

In [ ]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)